In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Google developer API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [2]:
# Create cities dataframe
cities_df = pd.read_csv('../WeatherPy/cities.csv')
cities_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,0,itaberai,-16.0203,-49.8103,71.31,74,12,4.00,BR,1634962556
1,1,puerto ayora,-0.7393,-90.3518,69.96,86,37,7.00,EC,1634962556
2,2,borba,-4.3878,-59.5939,76.84,80,97,2.42,BR,1634962557
3,3,phan thiet,10.9333,108.1000,86.49,65,91,2.75,VN,1634962505
4,4,iqaluit,63.7506,-68.5145,26.33,86,90,8.01,CA,1634962557


In [3]:
# Store latitude and longitude in locations
locations = cities_df[["Latitude", "Longitude"]]

# convert to float
humid = cities_df["Humidity (%)"].astype(float)

In [25]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
cities_df["Max Temperature (F)"]

0      71.31
1      69.96
2      76.84
3      86.49
4      26.33
       ...  
598    44.17
599    57.85
600    86.20
601    75.51
602    79.86
Name: Max Temperature (F), Length: 603, dtype: float64

In [13]:
#Create new dataframe looking for new criteria
vacationcities_df = cities_df

vacationcities_df = vacationcities_df.loc[cities_df["Max Temperature (F)"] < 80]
vacationcities_df = vacationcities_df.loc[cities_df["Max Temperature (F)"] > 70]
vacationcities_df = vacationcities_df.loc[cities_df["Wind Speed (mph)"] < 10]
vacationcities_df = vacationcities_df.loc[cities_df["Cloudiness (%)"] == 0]

#Drop the extra rows
vacationcities_df = vacationcities_df.dropna()

vacationcities_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
12,12,busselton,-33.6500,115.3333,70.09,53,0,6.76,AU,1634962559
53,56,hoshangabad,22.7500,77.7167,75.56,57,0,2.89,IN,1634962574
117,125,sao filipe,14.8961,-24.4956,79.56,69,0,3.27,CV,1634962598
163,178,vao,-22.6667,167.4833,73.60,77,0,5.82,NC,1634962614
179,195,salalah,17.0151,54.0924,77.09,73,0,2.30,OM,1634962624
207,226,topi,34.0704,72.6236,70.88,55,0,3.11,PK,1634962633
512,553,dera bugti,29.0307,69.1510,78.28,14,0,5.59,PK,1634962745
544,589,praia,14.9215,-23.5087,77.54,57,0,9.22,CV,1634962524


In [ ]:
hotel_df = vacationcities_df
hotel_df['Hotel Name'] = ""

In [11]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

for index, row in hotel_df.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    params['location'] = f"{latitude},{longitude}"
    
    print(f"Retrieving Results for Vacation Spot {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Nearest Hotel is {results[0]['name']}")
        hotel_df.loc[index, 'Hotel Name'] = results [0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    print("---------------------------------")

Retrieving Results for Vacation Spot 12: busselton.
Nearest Hotel is Observatory Guest House
---------------------------------
Retrieving Results for Vacation Spot 53: hoshangabad.
Nearest Hotel is NAMAN NARMADA
---------------------------------
Retrieving Results for Vacation Spot 117: sao filipe.
Nearest Hotel is Hotel Xaguate
---------------------------------
Retrieving Results for Vacation Spot 163: vao.
Nearest Hotel is Hotel Kou-Bugny
---------------------------------
Retrieving Results for Vacation Spot 179: salalah.
Nearest Hotel is HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel
---------------------------------
Retrieving Results for Vacation Spot 207: topi.
Nearest Hotel is Hostel 6
---------------------------------
Retrieving Results for Vacation Spot 512: dera bugti.
Nearest Hotel is Nawab Bugti House
---------------------------------
Retrieving Results for Vacation Spot 544: praia.
Nearest Hotel is LT Aparthotel
---------------------------------


In [12]:
hotel_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date,Hotel Name
12,12,busselton,-33.6500,115.3333,70.09,53,0,6.76,AU,1634962559,Observatory Guest House
53,56,hoshangabad,22.7500,77.7167,75.56,57,0,2.89,IN,1634962574,NAMAN NARMADA
117,125,sao filipe,14.8961,-24.4956,79.56,69,0,3.27,CV,1634962598,Hotel Xaguate
163,178,vao,-22.6667,167.4833,73.60,77,0,5.82,NC,1634962614,Hotel Kou-Bugny
179,195,salalah,17.0151,54.0924,77.09,73,0,2.30,OM,1634962624,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
207,226,topi,34.0704,72.6236,70.88,55,0,3.11,PK,1634962633,Hostel 6
512,553,dera bugti,29.0307,69.1510,78.28,14,0,5.59,PK,1634962745,Nawab Bugti House
544,589,praia,14.9215,-23.5087,77.54,57,0,9.22,CV,1634962524,LT Aparthotel


In [19]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
vacayspots = hotel_df[['Latitude', 'Longitude']]
markers = gmaps.marker_layer(vacayspots, info_box_content=hotel_info)

fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))